In [1]:
import cv2
import os
from multiprocessing import Pool
from datetime import datetime

def process_video_part(args):
    video, start_frame, end_frame = args
    vidcap = cv2.VideoCapture(video)

    vidcap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    while start_frame < end_frame and vidcap.isOpened():
        success, image = vidcap.read()
        if success:
            if start_frame % 3 == 0:  # Save every 3rd frame
                now = datetime.now().strftime("%d.%m.%y_%H:%M:%S.%f")[:-3]
                cv2.imwrite(os.path.join('dataset/images', f'{now}.png'), image)
            start_frame += 1
        else:
            break
    vidcap.release()

def video_to_frames_parallel(video, num_parts=2):
    vidcap = cv2.VideoCapture(video)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    vidcap.release()

    frames_list = []

    for i in range(1, num_parts+1):
        end_frame = int(total_frames * (i/num_parts))
        frames_list.append((video, frames_list[i-2][2] + 1 if i > 1 else 0, end_frame))

    with Pool(num_parts) as pl:
        pl.map(process_video_part, frames_list)

    cv2.destroyAllWindows()

video_to_frames_parallel('dataset/video/2023-11-09 13-16-57.mkv', num_parts=3)


3
[('dataset/video/2023-11-09 13-16-57.mkv', 0, 35917), ('dataset/video/2023-11-09 13-16-57.mkv', 35918, 71834), ('dataset/video/2023-11-09 13-16-57.mkv', 71835, 107751)]
